In [1]:
import sys,os
__script_path=os.path.abspath(globals().get('__file__','.'))
__script_dir = os.path.dirname(__script_path)
root_dir = os.path.abspath(f'{__script_dir}/../../..')
print(root_dir)
for lib in [root_dir][::-1]:
    if lib in sys.path:
        sys.path.remove(lib)
    sys.path.insert(0,lib)
from libs.common import *
from config.config import *

c:\Users\PC\Data\ads_fraud_detection


c:/Users/PC/Data/ads_fraud_detection


# Lựa chọn đặc trưng

### (1) Tạo thư mục lưu trữ (nếu chưa có)

In [2]:
save_dir=f"{exps_dir}/exp2/exp_gan"

os.makedirs(save_dir, exist_ok=True)
test_size=0.33
seed=42

In [3]:
x_train=pd.read_excel(f'{save_dir}/gen_x.xlsx')
y_train=pd.read_excel(f'{save_dir}/gen_y.xlsx')
x_test=pd.read_excel(f'{save_dir}/x_test.xlsx')
y_test=pd.read_excel(f'{save_dir}/y_test.xlsx')
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

(61511, 34) (61511, 1) (5089, 34) (5089, 1)


### (2) Lựa chọn các đặc trưng

In [4]:
x_train

,Month,WeekOfMonth,DayOfWeek,Make,AccidentArea,DayOfWeekClaimed,MonthClaimed,WeekOfMonthClaimed,Sex,MaritalStatus,Age,Fault,PolicyType,VehicleCategory,VehiclePrice,PolicyNumber,RepNumber,Deductible,DriverRating,Days_Policy_Accident,Days_Policy_Claim,PastNumberOfClaims,AgeOfVehicle,AgeOfPolicyHolder,PoliceReportFiled,WitnessPresent,AgentType,NumberOfSuppliments,AddressChange_Claim,NumberOfCars,Year,BasePolicy,week_claimed,week_incident
0,10,1,2,13,0,0,2,1,0,0,1.171392,0,4,0,1,0.016316,15,2,0,2,1,3,2,6,0,1,1,1,0,1,0,0,537,358
1,8,1,6,13,0,6,7,4,0,1,1.646604,0,3,0,1,-0.136182,6,2,0,2,2,0,2,3,0,1,0,0,1,1,0,1,848,986
2,9,3,4,13,0,4,10,1,0,0,2.377108,0,3,0,4,1.290072,9,2,0,2,1,2,1,5,0,0,1,0,1,2,1,0,355,595
3,1,0,5,13,0,2,3,3,0,0,0.951542,0,4,0,0,0.125356,9,3,0,2,2,2,2,7,0,1,0,1,1,0,0,2,580,51
4,2,0,1,13,0,4,5,1,0,0,0.758361,1,5,0,3,0.732805,5,2,2,2,2,2,1,5,0,0,1,1,0,0,2,0,98,464
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61506,3,3,6,16,1,6,3,3,1,2,-0.433660,0,2,1,2,-0.160616,15,1,3,3,2,1,4,4,0,0,0,3,3,0,1,2,411,516
61507,6,2,1,16,1,6,1,3,1,1,0.008896,0,2,1,3,0.797519,0,1,3,3,2,1,5,5,0,0,0,3,3,0,1,2,357,601
61508,5,2,5,16,1,4,1,0,1,1,1.336565,1,0,0,2,-1.159307,6,1,1,3,2,1,6,7,0,0,0,1,3,0,0,0,28,180
61509,1,3,0,16,1,0,11,4,1,1,2.590474,0,0,0,3,0.138073,0,1,1,3,2,0,6,8,0,0,0,3,3,0,1,0,616,444


In [5]:
def modelcheck_info(model,name):
    print(name)
    try:
        coefficients = model.coef_.flatten()
    except:
        coefficients = model.feature_importances_.flatten()
    feature_importance = pd.DataFrame({
        'Feature': x_train.columns,
        'Coefficient': coefficients
    })

    print('bi loai bo',feature_importance[feature_importance['Coefficient'] == 0])
    feature_importance = feature_importance[feature_importance['Coefficient'] != 0]
    feature_importance['Absolute Coefficient'] = feature_importance['Coefficient'].abs()
    feature_importance.sort_values(by='Absolute Coefficient', ascending=False, inplace=True)

    print("Xếp hạng các đặc trưng theo mức độ quan trọng:")
    print(feature_importance[['Feature', 'Coefficient']])

In [6]:
models=[]
models.append(LogisticRegression())
models.append(LinearDiscriminantAnalysis())
models.append(DecisionTreeClassifier())
names=['LR','LDA','DTC']
feature_SelectFromModel={}
for model,name in zip(models,names):
    check = model.fit(x_train, y_train)
    selector = SelectFromModel(check, prefit=True)

    selected_features = x_train.columns[(selector.get_support())]
    modelcheck_info(model,name)
    print("Các đặc trưng được chọn:")
    feature_SelectFromModel[name]=list(selected_features)
    print(selected_features)

LR
bi loai bo Empty DataFrame
Columns: [Feature, Coefficient]
Index: []
Xếp hạng các đặc trưng theo mức độ quan trọng:
                 Feature  Coefficient
28   AddressChange_Claim    -0.132801
27   NumberOfSuppliments    -0.130902
11                 Fault    -0.116359
4           AccidentArea    -0.092640
20     Days_Policy_Claim    -0.088291
18          DriverRating    -0.084619
19  Days_Policy_Accident    -0.083623
2              DayOfWeek    -0.070796
12            PolicyType    -0.065863
30                  Year    -0.063679
31            BasePolicy    -0.061344
8                    Sex    -0.051234
25        WitnessPresent     0.047586
22          AgeOfVehicle    -0.043612
17            Deductible    -0.040083
29          NumberOfCars    -0.038683
1            WeekOfMonth    -0.032092
14          VehiclePrice     0.027837
26             AgentType    -0.025318
9          MaritalStatus    -0.021104
0                  Month    -0.020794
15          PolicyNumber     0.015411
6      

### Giải thích:

Phương pháp **SelectFromModel** từ thư viện sklearn.feature_selection. Phương pháp này hoạt động bằng cách sử dụng một mô hình học máy để đánh giá độ quan trọng của từng đặc trưng và sau đó chọn những đặc trưng quan trọng nhất.

   Các model được dùng trong phương pháp này:
   * **'LR'**: LogisticRegression
   * **'LDA'**: LinearDiscriminantAnalysis
   * **'DTC'**: DecisionTreeClassifier
  
       #### 1. LogisticRegression:
       Các đặc trưng được chọn:
* Index(['age', 'policy_csl', 'insured_education_level', 'insured_occupation',
       'incident_severity', 'incident_state', 'incident_city',
       'incident_hour_of_the_day', 'property_damage', 'witnesses',
       'property_claim', 'vehicle_claim', 'auto_make'],
      dtype='object')
      
   #### 2. LinearDiscriminantAnalysis:
   Các đặc trưng được chọn:
* Index(['policy_number', 'policy_annual_premium', 'insured_zip', 'insured_sex',
       'incident_severity', 'incident_hour_of_the_day', 'property_damage',
       'total_claim_amount', 'injury_claim', 'property_claim',
       'vehicle_claim'],
      dtype='object')
      
   #### 3. DecisionTreeClassifier:
   Các đặc trưng được chọn:
* Index(['policy_number', 'policy_annual_premium', 'insured_zip',
       'insured_occupation', 'insured_hobbies', 'incident_severity',
       'incident_location', 'police_report_available', 'total_claim_amount',
       'injury_claim', 'month_policy_bind_date', 'day_incident_date'],
      dtype='object')

In [7]:
def feature_selection_inRFE(model,name):
    rfe=RFE(estimator=model, n_features_to_select=10)
    fit=rfe.fit(x_train, y_train)
#     print("Num Features: %d"% fit.n_features_)
#     print("Selected Features: %s" % fit.support_)
#     print("Features Ranking: %s" %fit.ranking_)
#     for index, value in enumerate(fit.ranking_):
#         print(f"{value}: {x_train.columns[index]}")
    sorted_columns = [x for _, x in sorted(zip(fit.ranking_, x_train.columns))]
    sort=sorted(fit.ranking_)

#     for i,j in zip(sort,sorted_columns):
#         print("Columns sorted by RFE ranking",i,":",j)

    return [j for i,j in zip(sort,sorted_columns) if i==1]

In [8]:
models = [
    LogisticRegression(),
    LinearDiscriminantAnalysis(),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    LGBMClassifier(),  
    XGBClassifier(use_label_encoder=False)  
]
feature_RFE={}

names = ['LR', 'LDA', 'DTC', 'RD', 'LGBM', 'XGB']
for model,name in zip(models,names):
    print("RFE")
    print(f"Kiểm tra mô hình {name}")
    model.fit(x_train,y_train)
    print(feature_selection_inRFE(model,name))
    feature_RFE[name]=feature_selection_inRFE(model,name)

RFE
Kiểm tra mô hình LR
['AccidentArea', 'AddressChange_Claim', 'AgentType', 'Days_Policy_Claim', 'Fault', 'NumberOfCars', 'NumberOfSuppliments', 'Sex', 'WitnessPresent', 'Year']
RFE
Kiểm tra mô hình LDA
['AccidentArea', 'AddressChange_Claim', 'AgentType', 'BasePolicy', 'Days_Policy_Claim', 'Fault', 'NumberOfCars', 'NumberOfSuppliments', 'WitnessPresent', 'Year']
RFE
Kiểm tra mô hình DTC
['Age', 'Make', 'Month', 'MonthClaimed', 'PolicyNumber', 'RepNumber', 'WeekOfMonth', 'WeekOfMonthClaimed', 'week_claimed', 'week_incident']
RFE
Kiểm tra mô hình RD
['Age', 'AgeOfPolicyHolder', 'DayOfWeekClaimed', 'Make', 'Month', 'MonthClaimed', 'PolicyNumber', 'RepNumber', 'week_claimed', 'week_incident']
RFE
Kiểm tra mô hình LGBM
[LightGBM] [Info] Number of positive: 9783, number of negative: 51728
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002729 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `f

### Giải thích:
**Recursive Feature Elimination (RFE)** là một phương pháp trong lựa chọn đặc trưng (feature selection) được sử dụng trong học máy để giảm số lượng đặc trưng của mô hình mà vẫn giữ được hiệu suất tốt.

Các model được dùng trong phương pháp này:
   * **'LR'**: LogisticRegression
   * **'LDA'**: LinearDiscriminantAnalysis
   * **'DTC'**: DecisionTreeClassifier
   * **'RD'**: RandomForestClassifier

In [9]:
special_models=[KNeighborsClassifier(),GaussianNB(),MLPClassifier(),GradientBoostingClassifier(),MultinomialNB(),SVC()]
names=['KNC','GNB','NN','BG','NB','SVM']
def feature_selection_inKBest(model, name):
    k_best = SelectKBest(score_func=f_classif, k=11)
    fit = k_best.fit(x_train, y_train)
    selected_features = fit.get_support()
    selected_column_names = x_train.columns[selected_features]
    return selected_column_names
feature_Kbest={}
for model,name in zip(special_models,names):
    print(f'Lua chon dac trung {name}')
    print(feature_selection_inKBest(model, name))
    feature_Kbest[name]=list(feature_selection_inKBest(model, name))

Lua chon dac trung KNC
Index(['DayOfWeek', 'AccidentArea', 'Sex', 'Fault', 'DriverRating', 'Days_Policy_Claim', 'AgeOfVehicle', 'WitnessPresent', 'NumberOfSuppliments', 'AddressChange_Claim', 'Year'], dtype='object')
Lua chon dac trung GNB
Index(['DayOfWeek', 'AccidentArea', 'Sex', 'Fault', 'DriverRating', 'Days_Policy_Claim', 'AgeOfVehicle', 'WitnessPresent', 'NumberOfSuppliments', 'AddressChange_Claim', 'Year'], dtype='object')
Lua chon dac trung NN
Index(['DayOfWeek', 'AccidentArea', 'Sex', 'Fault', 'DriverRating', 'Days_Policy_Claim', 'AgeOfVehicle', 'WitnessPresent', 'NumberOfSuppliments', 'AddressChange_Claim', 'Year'], dtype='object')
Lua chon dac trung BG
Index(['DayOfWeek', 'AccidentArea', 'Sex', 'Fault', 'DriverRating', 'Days_Policy_Claim', 'AgeOfVehicle', 'WitnessPresent', 'NumberOfSuppliments', 'AddressChange_Claim', 'Year'], dtype='object')
Lua chon dac trung NB
Index(['DayOfWeek', 'AccidentArea', 'Sex', 'Fault', 'DriverRating', 'Days_Policy_Claim', 'AgeOfVehicle', 'Witnes

### Giải thích:
Phương pháp **SelectKBest** từ thư viện sklearn.feature_selection để lựa chọn đặc trưng dựa trên kỹ thuật kiểm định thống kê.
 
Các model được dùng trong phương pháp này:
   * **'KNC'**: KNeighborsClassifier
   * **'GNB'**: GaussianNB
   * **'NN'**: MLPClassifier
   * **'BG'**: GradientBoostingClassifier
   * **'NB'**: MultinomialNB
   * **'SVM'**: SVC

In [10]:
print(feature_Kbest)
print(feature_SelectFromModel)
print(feature_RFE)

{'KNC': ['DayOfWeek', 'AccidentArea', 'Sex', 'Fault', 'DriverRating', 'Days_Policy_Claim', 'AgeOfVehicle', 'WitnessPresent', 'NumberOfSuppliments', 'AddressChange_Claim', 'Year'], 'GNB': ['DayOfWeek', 'AccidentArea', 'Sex', 'Fault', 'DriverRating', 'Days_Policy_Claim', 'AgeOfVehicle', 'WitnessPresent', 'NumberOfSuppliments', 'AddressChange_Claim', 'Year'], 'NN': ['DayOfWeek', 'AccidentArea', 'Sex', 'Fault', 'DriverRating', 'Days_Policy_Claim', 'AgeOfVehicle', 'WitnessPresent', 'NumberOfSuppliments', 'AddressChange_Claim', 'Year'], 'BG': ['DayOfWeek', 'AccidentArea', 'Sex', 'Fault', 'DriverRating', 'Days_Policy_Claim', 'AgeOfVehicle', 'WitnessPresent', 'NumberOfSuppliments', 'AddressChange_Claim', 'Year'], 'NB': ['DayOfWeek', 'AccidentArea', 'Sex', 'Fault', 'DriverRating', 'Days_Policy_Claim', 'AgeOfVehicle', 'WitnessPresent', 'NumberOfSuppliments', 'AddressChange_Claim', 'Year'], 'SVM': ['DayOfWeek', 'AccidentArea', 'Sex', 'Fault', 'DriverRating', 'Days_Policy_Claim', 'AgeOfVehicle', '

In [11]:
feature_model_selection={**feature_RFE,**feature_Kbest}
np.savez(f'{save_dir}/feature_model_selection.npz', feature_model_selection = feature_model_selection)
feature_model_selection

{'LR': ['AccidentArea',
  'AddressChange_Claim',
  'AgentType',
  'Days_Policy_Claim',
  'Fault',
  'NumberOfCars',
  'NumberOfSuppliments',
  'Sex',
  'WitnessPresent',
  'Year'],
 'LDA': ['AccidentArea',
  'AddressChange_Claim',
  'AgentType',
  'BasePolicy',
  'Days_Policy_Claim',
  'Fault',
  'NumberOfCars',
  'NumberOfSuppliments',
  'WitnessPresent',
  'Year'],
 'DTC': ['Age',
  'Make',
  'Month',
  'MonthClaimed',
  'PolicyNumber',
  'RepNumber',
  'WeekOfMonth',
  'WeekOfMonthClaimed',
  'week_claimed',
  'week_incident'],
 'RD': ['Age',
  'AgeOfPolicyHolder',
  'DayOfWeekClaimed',
  'Make',
  'Month',
  'MonthClaimed',
  'PolicyNumber',
  'RepNumber',
  'week_claimed',
  'week_incident'],
 'LGBM': ['Age',
  'AgeOfVehicle',
  'Make',
  'Month',
  'MonthClaimed',
  'PolicyNumber',
  'PolicyType',
  'RepNumber',
  'week_claimed',
  'week_incident'],
 'XGB': ['AccidentArea',
  'AddressChange_Claim',
  'AgentType',
  'BasePolicy',
  'Days_Policy_Claim',
  'Fault',
  'NumberOfCars',

* gom nhóm thông tin về các đặc trưng được chọn từ hai phương pháp lựa chọn đặc trưng khác nhau (RFE và SelectKBest) vào từ điển **feature_model_selection**.

In [12]:
dict(np.load(f'{save_dir}/feature_model_selection.npz',allow_pickle=True))['feature_model_selection']

array({'LR': ['AccidentArea', 'AddressChange_Claim', 'AgentType', 'Days_Policy_Claim', 'Fault', 'NumberOfCars', 'NumberOfSuppliments', 'Sex', 'WitnessPresent', 'Year'], 'LDA': ['AccidentArea', 'AddressChange_Claim', 'AgentType', 'BasePolicy', 'Days_Policy_Claim', 'Fault', 'NumberOfCars', 'NumberOfSuppliments', 'WitnessPresent', 'Year'], 'DTC': ['Age', 'Make', 'Month', 'MonthClaimed', 'PolicyNumber', 'RepNumber', 'WeekOfMonth', 'WeekOfMonthClaimed', 'week_claimed', 'week_incident'], 'RD': ['Age', 'AgeOfPolicyHolder', 'DayOfWeekClaimed', 'Make', 'Month', 'MonthClaimed', 'PolicyNumber', 'RepNumber', 'week_claimed', 'week_incident'], 'LGBM': ['Age', 'AgeOfVehicle', 'Make', 'Month', 'MonthClaimed', 'PolicyNumber', 'PolicyType', 'RepNumber', 'week_claimed', 'week_incident'], 'XGB': ['AccidentArea', 'AddressChange_Claim', 'AgentType', 'BasePolicy', 'Days_Policy_Claim', 'Fault', 'NumberOfCars', 'PolicyType', 'WitnessPresent', 'Year'], 'KNC': ['DayOfWeek', 'AccidentArea', 'Sex', 'Fault', 'Drive